In [1]:
radio_number = 80

In [2]:
# This is an example from the Crazyflie Python API.
# See https://github.com/bitcraze/crazyflie-lib-python/blob/master/examples/basiclogSync.py

import logging
import time

import cflib.crtp
from cflib.crazyflie import Crazyflie
from cflib.crazyflie.log import LogConfig
from cflib.crazyflie.syncCrazyflie import SyncCrazyflie
from cflib.crazyflie.syncLogger import SyncLogger

# Only output errors from the logging framework
logging.basicConfig(level=logging.ERROR)


# Initialize the low-level drivers (don't list the debug drivers)
cflib.crtp.init_drivers(enable_debug_driver=False)
# Scan for Crazyflies and use the first one found
print('Scanning interfaces for Crazyflies...')
available = cflib.crtp.scan_interfaces()
print('Crazyflies found:')
for i in available:
    print(i[0])

if len(available) == 0:
    print('No Crazyflies found, cannot run example')
else:
    lg_stab = LogConfig(name='Stabilizer', period_in_ms=10)
    lg_stab.add_variable('stabilizer.roll', 'float')
    lg_stab.add_variable('stabilizer.pitch', 'float')
    lg_stab.add_variable('stabilizer.yaw', 'float')

    cf = Crazyflie(rw_cache='./cache')
    with SyncCrazyflie(available[0][0], cf=cf) as scf:
        with SyncLogger(scf, lg_stab) as logger:
            endTime = time.time() + 10

            for log_entry in logger:
                timestamp = log_entry[0]
                data = log_entry[1]
                logconf_name = log_entry[2]

                print('[%d][%s]: %s' % (timestamp, logconf_name, data))

                if time.time() > endTime:
                    break

Scanning interfaces for Crazyflies...
Crazyflies found:
radio://0/80/2M
[256193][<cflib.crazyflie.log.LogConfig object at 0x10701be90>]: {'stabilizer.roll': 0.680008590221405, 'stabilizer.pitch': 0.3107483983039856, 'stabilizer.yaw': -0.6504475474357605}
[256203][<cflib.crazyflie.log.LogConfig object at 0x10701be90>]: {'stabilizer.roll': 0.8432763814926147, 'stabilizer.pitch': 0.12762416899204254, 'stabilizer.yaw': -0.6897426843643188}
[256213][<cflib.crazyflie.log.LogConfig object at 0x10701be90>]: {'stabilizer.roll': 0.7277297377586365, 'stabilizer.pitch': 0.17694026231765747, 'stabilizer.yaw': -0.674083948135376}
[256223][<cflib.crazyflie.log.LogConfig object at 0x10701be90>]: {'stabilizer.roll': 0.7385603189468384, 'stabilizer.pitch': 0.12084020674228668, 'stabilizer.yaw': -0.6863424777984619}
[256233][<cflib.crazyflie.log.LogConfig object at 0x10701be90>]: {'stabilizer.roll': 0.7563152313232422, 'stabilizer.pitch': 0.12507420778274536, 'stabilizer.yaw': -0.6860293745994568}
[25624

In [3]:
DEFAULT_HEIGHT = 0.5

def wait_for_position_estimator(scf):
    print('Waiting for estimator to find position...')
    
    time.sleep(1)

    log_config = LogConfig(name='Kalman Variance', period_in_ms=500)
    log_config.add_variable('kalman.varPX', 'float')
    log_config.add_variable('kalman.varPY', 'float')
    log_config.add_variable('kalman.varPZ', 'float')

    var_y_history = [1000] * 10
    var_x_history = [1000] * 10
    var_z_history = [1000] * 10

    threshold = 0.001
    with SyncLogger(scf, log_config) as logger:
        for log_entry in logger:
            data = log_entry[1]

            var_x_history.append(data['kalman.varPX'])
            var_x_history.pop(0)
            var_y_history.append(data['kalman.varPY'])
            var_y_history.pop(0)
            var_z_history.append(data['kalman.varPZ'])
            var_z_history.pop(0)

            min_x = min(var_x_history)
            max_x = max(var_x_history)
            min_y = min(var_y_history)
            max_y = max(var_y_history)
            min_z = min(var_z_history)
            max_z = max(var_z_history)

            print("{} {} {}".
                format(max_x - min_x, max_y - min_y, max_z - min_z))

            if (max_x - min_x) < threshold and (
                    max_y - min_y) < threshold and (
                    max_z - min_z) < threshold:
                break

# Initialize controller: 
def set_PID_controller(cf):
    # Set the PID Controller:
    print('Initializing PID Controller')
    #cf.param.set_value('stabilizer.controller', '1')
    cf.param.set_value('kalman.resetEstimation', '1')
    time.sleep(0.1)
    cf.param.set_value('kalman.resetEstimation', '0')
    
    wait_for_position_estimator(cf)
    time.sleep(0.1)    
    return

# Ascend and hover: 
def ascend_and_hover(cf):
    # Ascend:
    for y in range(20):
        cf.commander.send_hover_setpoint(0, 0, 0, y / 50)
        time.sleep(0.1)
    # Hover at 0.5 meters:
    for _ in range(30):
        cf.commander.send_hover_setpoint(0, 0, 0, 0.5)
        time.sleep(0.1)
    return
    
def move_forward_and_backward(cf):
    
    print('moving right')
    for _ in range(15):
        cf.commander.send_hover_setpoint(0, 0.5, 0, 0.5)
        time.sleep(0.1)
    
    print('moving left')
    for _ in range(15):
        cf.commander.send_hover_setpoint(0, -0.5, 0, 0.5)
        time.sleep(0.1)
        
    print('moving left')
    for _ in range(15):
        cf.commander.send_hover_setpoint(0, -0.5, 0, 0.5)
        time.sleep(0.1)
        
    print('moving right')
    for _ in range(15):
        cf.commander.send_hover_setpoint(0, 0.5, 0, 0.5)
        time.sleep(0.1)
        
    print('moving forward')
    for _ in range(22):
        cf.commander.send_hover_setpoint(1.0, 0, 0, 0.5)
        time.sleep(0.1)
    
    print('turning')
    for _ in range(30):
        cf.commander.send_hover_setpoint(0, 0, 45, 0.5)
        time.sleep(0.1)
        
    
    print('moving forward')
    #for _ in range(22):
     #   cf.commander.send_hover_setpoint(0.8, 0, 20, 0.5)
      #  time.sleep(0.1)
    return

def ascend_and_hover_at_one_m(cf):
    # Ascend:
    for y in range(50,70,1):
        cf.commander.send_hover_setpoint(0, 0, 0, y / 100)
        time.sleep(0.1)
    # Hover at 0.5 meters:
    for _ in range(30):
        cf.commander.send_hover_setpoint(0, 0, 0, 1)
        time.sleep(0.1)
        
    for y in range(5):
        cf.commander.send_hover_setpoint(0, 0, 0, (10 - y) / 10)
        time.sleep(0.1)
    return

# Hover, descend, and stop all motion:
def hover_and_descend(cf):
    # Hover at 0.5 meters:
    for _ in range(30):
        cf.commander.send_hover_setpoint(0, 0, 0, 0.5)
        time.sleep(0.1)
    # Descend:
    for y in range(10):
        cf.commander.send_hover_setpoint(0, 0, 0, (10 - y) / 25)
        time.sleep(0.1)
    # Stop all motion:
    for i in range(10):
        cf.commander.send_stop_setpoint()
        time.sleep(0.1)
    return

def spiral_upwards(cf):
    cf.commander.send_hover_setpoint(0, 0, 0, 0.05)
    time.sleep(0.1)
    # Ascend:
    for y in range(190):
        cf.commander.send_hover_setpoint(0.4, 0, 58, y / 100)
        time.sleep(0.1)
    # Hover at 0.5 meters:
    for _ in range(30):
        cf.commander.send_hover_setpoint(0, 0, 0, 2.0)
        time.sleep(0.1)
    # Descend:
    for y in range(20):
        cf.commander.send_hover_setpoint(0, 0, 0, (20 - y) / 10)
        time.sleep(0.1)
    # Stop all motion:
    for i in range(10):
        cf.commander.send_stop_setpoint()
        time.sleep(0.1)
    return

In [4]:
setpoint_delay = 3 # (int) Time to give the controller to reach next setpoint; in multiples of 0.1 s
    
# Set the URI the Crazyflie will connect to
uri = f'radio://0/{radio_number}/2M'
    
# Initialize all the CrazyFlie drivers:
cflib.crtp.init_drivers(enable_debug_driver=False)

# Sync to the CrazyFlie: 
with SyncCrazyflie(uri, cf=Crazyflie(rw_cache='./cache')) as scf:
    # Get the Crazyflie class instance:
    cf = scf.cf

    # Initialize and ascend:
    set_PID_controller(cf)
    ascend_and_hover(cf)
    
    # fly forward
    #move_forward_and_backward(cf)
    
    # ascend to one meter and descend again
    #ascend_and_hover_at_one_m(cf)

    # Descend and stop all motion: 
    hover_and_descend(cf)
            
print('Done!') 

Initializing PID Controller
Waiting for estimator to find position...
999.9999006666039 999.9999006441212 999.9998348445079
999.9999050771366 999.9999050420665 999.9998348445079
999.9999050771366 999.9999050420665 999.9998348445079
999.9999050771366 999.9999050420665 999.9998348445079
999.9999050771366 999.9999050420665 999.9998348445079
999.9999050771366 999.9999050420665 999.9998369066307
999.9999050771366 999.9999050420665 999.9998369066307
999.9999050771366 999.9999050420665 999.9998369066307
999.9999050771366 999.9999050420665 999.9998369066307
1.6435507859569043e-05 1.671331119723618e-05 5.168482312001288e-05
Done!


In [7]:
setpoint_delay = 3 # (int) Time to give the controller to reach next setpoint; in multiples of 0.1 s
    
# Set the URI the Crazyflie will connect to
uri = f'radio://0/{radio_number}/2M'
    
# Initialize all the CrazyFlie drivers:
cflib.crtp.init_drivers(enable_debug_driver=False)

# Sync to the CrazyFlie: 
with SyncCrazyflie(uri, cf=Crazyflie(rw_cache='./cache')) as scf:
    # Get the Crazyflie class instance:
    cf = scf.cf

    # Initialize and ascend:
    set_PID_controller(cf)
    spiral_upwards(cf)
            
print('Done!')

Initializing PID Controller
Waiting for estimator to find position...
999.9998887199326 999.999888720391 999.9998027741385
999.9998887199326 999.999888720391 999.9998213555373
999.9998887199326 999.999888720391 999.9998213555373
999.9998887199326 999.999888720391 999.9998213555373
999.9998926959524 999.9998926961125 999.9998237553809
999.9998926959524 999.9998926961125 999.9998237553809
999.9998926959524 999.9998926961125 999.9998237553809
999.9998926959524 999.9998926961125 999.9998237553809
999.9998942306192 999.9998942307502 999.9998237553809
2.8029280656483024e-05 2.8029164241161197e-05 3.232475137338042e-05


KeyboardInterrupt: 